In [1]:
import rasterio
import numpy as np
import joblib
from rasterio.plot import reshape_as_image
import pandas as pd
import os

In [9]:
raster_dir = r"C:\Penelitian S3\Analisis Perubahan Lahan\Landsat 8\2024"
texture_dir = r"C:\Penelitian S3\Analisis Perubahan Lahan\Landsat 8\2024"
model_path = r"C:\Penelitian S3\Analisis Perubahan Lahan\rf_model_landsat89.pkl"
output_path = r"C:\Penelitian S3\Analisis Perubahan Lahan\output lulc\classified_lulc_2024_rf.tif"

In [10]:
slope_path = r"C:\Penelitian S3\Analisis Perubahan Lahan\Spatial Data\slope30m_resampled.tif"
aspect_path = r"C:\Penelitian S3\Analisis Perubahan Lahan\Spatial Data\aspect30m_resampled.tif"

In [11]:
raster_files = {
    "SR": "L8_2024_SR_Bands.tif",
    "NDVI": "L8_2024_NDVI.tif",
    "EVI": "L8_2024_EVI.tif",
    "SAVI": "L8_2024_SAVI.tif",
    "NDWI": "L8_2024_NDWI.tif",
    "NDBI": "L8_2024_NDBI.tif",
    "NDMI": "L8_2024_NDMI.tif",
    "BSI": "L8_2024_BSI.tif",
    "MNDWI": "L8_2024_MNDWI.tif",
}

In [12]:
texture_files = [
    "output67e191d23d8342118d844d0c8905a119_ASM.tif",
    "output67e191d23d8342118d844d0c8905a119_Contr.tif",
    "output67e191d23d8342118d844d0c8905a119_Corr.tif",
    "output67e191d23d8342118d844d0c8905a119_Entr.tif",
    "output67e191d23d8342118d844d0c8905a119_IDM.tif",
    "output67e191d23d8342118d844d0c8905a119_Var.tif"
]
texture_names = ["ASM", "Contrast", "Correlation", "Entropy", "IDM", "Variance"]

In [13]:
bands_list = []

In [14]:
with rasterio.open(os.path.join(raster_dir, raster_files["SR"])) as src:
    sr_data = src.read([1, 2, 3, 4, 5, 6])
    profile = src.profile
    height, width = src.height, src.width
    bands_list.extend([sr_data[i, :, :] for i in range(sr_data.shape[0])])

In [15]:
for idx in ["NDVI", "EVI", "SAVI", "NDWI", "NDBI", "NDMI", "BSI", "MNDWI"]:
    with rasterio.open(os.path.join(raster_dir, raster_files[idx])) as src:
        bands_list.append(src.read(1))

In [16]:
for path in [slope_path, aspect_path]:
    with rasterio.open(path) as src:
        bands_list.append(src.read(1))

In [17]:
for fname in texture_files:
    with rasterio.open(os.path.join(texture_dir, fname)) as src:
        bands_list.append(src.read(1))

In [18]:
print("\nCek shape semua band:")
for i, band in enumerate(bands_list):
    print(f"Band {i+1}: shape = {band.shape}")


Cek shape semua band:
Band 1: shape = (3000, 2242)
Band 2: shape = (3000, 2242)
Band 3: shape = (3000, 2242)
Band 4: shape = (3000, 2242)
Band 5: shape = (3000, 2242)
Band 6: shape = (3000, 2242)
Band 7: shape = (3000, 2242)
Band 8: shape = (3000, 2242)
Band 9: shape = (3000, 2242)
Band 10: shape = (3000, 2242)
Band 11: shape = (3000, 2242)
Band 12: shape = (3000, 2242)
Band 13: shape = (3000, 2242)
Band 14: shape = (3000, 2242)
Band 15: shape = (3000, 2242)
Band 16: shape = (3000, 2242)
Band 17: shape = (3000, 2242)
Band 18: shape = (3000, 2242)
Band 19: shape = (3000, 2242)
Band 20: shape = (3000, 2242)
Band 21: shape = (3000, 2242)
Band 22: shape = (3000, 2242)


In [19]:
stacked = np.stack(bands_list, axis=0)
reshaped = reshape_as_image(stacked)
flat_pixels = reshaped.reshape(-1, reshaped.shape[-1])

In [20]:
feature_names = [
    "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7",
    "NDVI", "EVI", "SAVI", "NDWI", "NDBI", "NDMI", "BSI", "MNDWI",
    "Slope", "Aspect"
] + texture_names

df = pd.DataFrame(flat_pixels, columns=feature_names)

In [21]:
df["NDWI_IDM"] = df["NDWI"] / (df["IDM"] + 1e-6)
df["MNDWI_Contrast"] = df["MNDWI"] * df["Contrast"]
df["NDMI_Entropy"] = df["NDMI"] * df["Entropy"]

In [22]:
flat_pixels_extended = df.to_numpy()

In [23]:
print("\n Loading model dan klasifikasi...")
valid_mask = ~np.any(np.isnan(flat_pixels_extended), axis=1)

model = joblib.load(model_path)
predicted = np.full(flat_pixels_extended.shape[0], 0, dtype=np.uint8)
predicted[valid_mask] = model.predict(flat_pixels_extended[valid_mask])


 Loading model dan klasifikasi...


C:\Users\ASUS\miniconda3\envs\rc_env\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [24]:
classified = predicted.reshape(height, width)

In [25]:
profile.update(dtype=rasterio.uint8, count=1, compress='lzw', BIGTIFF='YES')
os.makedirs(os.path.dirname(output_path), exist_ok=True)
with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(classified.astype(rasterio.uint8), 1)

print(f"\n Raster klasifikasi disimpan di:\n{output_path}")


 Raster klasifikasi disimpan di:
C:\Penelitian S3\Analisis Perubahan Lahan\output lulc\classified_lulc_2024_rf.tif
